In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
data = '../input/breast-cancer-wisconsin-data/data.csv'

In [ ]:
table = pd.read_csv(data)

In [ ]:
table = table.sample(frac = 1)

In [ ]:
table.shape

In [ ]:
table.head()

In [ ]:
table.drop('Unnamed: 32', axis=1, inplace=True)

In [ ]:
table.drop('id', axis=1, inplace=True)

In [ ]:
ax = sns.countplot(table['diagnosis'],label="Count")
B, M = table['diagnosis'].value_counts()
print('Number of Benign: ',B)
print('Number of Malignant : ',M)

In [ ]:
from sklearn.preprocessing import  MinMaxScaler
scl=MinMaxScaler()

In [ ]:
table['diagnosis'].replace({'M':1, 'B':0}, inplace=True)

In [ ]:
test_set = table.iloc[-20:, :]

In [ ]:
test_set.head()

In [ ]:
drop_index = test_set.index

In [ ]:
drop_index

In [ ]:
table.drop(drop_index, inplace=True)

In [ ]:
table.shape

In [ ]:
x = table.iloc[:,1:]

In [ ]:
y = table.iloc[:,:1]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [ ]:
x_train=scl.fit_transform(x_train)
x_test=scl.fit_transform(x_test)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE()

In [ ]:
x_train_f, y_train_f = sm.fit_sample(x_train, y_train)
x_test_f, y_test_f = sm.fit_sample(x_test, y_test)

In [ ]:
print(x_train_f.shape, y_train_f.shape, x_test_f.shape, y_test_f.shape)

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.optimizers import Adam, Nadam, SGD, Adamax
from tensorflow.keras.models import Sequential

In [ ]:
act = ['relu', 'swish']
lr = 0.001
opt = [Adam(learning_rate=lr),SGD(learning_rate=lr),Nadam(learning_rate=lr), Adamax(learning_rate=lr)]
epoch = 200
optim = opt[3]
activ=act[1]

In [ ]:
def the_model():
    model = Sequential([
        L.Dense(x_train.shape[1], input_shape= (x_train.shape[1],)),
        L.BatchNormalization(),
        L.Dropout(0.3),
        L.Dense(50, activation=activ),
        L.BatchNormalization(),
        L.Dropout(0.3),
        L.Dense(50, activation=activ),
        L.BatchNormalization(),
        L.Dropout(0.2),
        L.Dense(20, activation=activ),
        L.BatchNormalization(),
        L.Dropout(0.2),
        L.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer= optim, loss='binary_crossentropy',metrics='accuracy')
    return model

In [ ]:
model = the_model()
hist = model.fit(x_train, y_train, epochs=epoch, validation_data=(x_test,y_test))

In [ ]:
model1 = the_model()
hist1 = model1.fit(x_train_f, y_train_f, epochs=epoch, validation_data=(x_test_f,y_test_f))

In [ ]:
pd.DataFrame(hist.history).plot(figsize=(18, 5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
pd.DataFrame(hist1.history).plot(figsize=(18, 5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
pd.DataFrame(hist.history).tail()

In [ ]:
pd.DataFrame(hist1.history).tail()

# Prediction

In [ ]:
valid_x = scl.fit_transform(test_set.iloc[:,1:])

In [ ]:
valid_y=test_set.iloc[:,:1]

In [ ]:
print(valid_x.shape, valid_y.shape)

In [ ]:
print(valid_y)

In [ ]:
prediction = model.predict(valid_x)

In [ ]:
prediction1 = model1.predict(valid_x)

In [ ]:
for i,j in zip(prediction, valid_y.values):
    if i >= 0.5:
        print('Malignant',j)
    else: print('Fine',j)

In [ ]:
for i,j in zip(prediction1, valid_y.values):
    if i >= 0.5:
        print('Malignant',j)
    else: print('Fine',j)